In [1]:
import numpy as np
import pandas as pd
from regex import search

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time 

import warnings
warnings.filterwarnings("ignore")

### Reading in artist data

In [2]:
# Reading in full dataset (includes wikipedia and spotify data), one row per group member
df = pd.read_csv('wiki_and_spoti_merged.csv')
df['clean_genre'] =list(map(lambda x: x[14:-6] if (str(x)[0:5] =='/wiki') else x, df.genre))
print(df.shape)
df.head()

(19362, 35)


,Unnamed: 0,genre,member,attr,band,years active,origin,band_no_parantheses,Group_name_(wiki),Group_name_(spoti)_x,...,mean_song_acousticness,mean_song_energy,mean_song_instrumentalness,mean_song_liveness,mean_song_loudness,mean_song_speechiness,mean_song_tempo,song_popularity_range,unwanted_member,clean_genre
0,0,/wiki/List_of_Christian_metal_bands,Angelika Korszyńska-Górny,current,"2Tm2,3",1996–present,Poland,"2Tm2,3","2Tm2,3","2TM2,3",...,0.122266,0.803,0.23076,0.21861,-9.7403,0.06528,143.5114,7.0,False,Christian_metal
1,1,/wiki/List_of_Christian_metal_bands,Tomasz Budzyński,current,"2Tm2,3",1996–present,Poland,"2Tm2,3","2Tm2,3","2TM2,3",...,0.122266,0.803,0.23076,0.21861,-9.7403,0.06528,143.5114,7.0,False,Christian_metal
2,2,/wiki/List_of_Christian_metal_bands,Mateusz Pospieszalski,current,"2Tm2,3",1996–present,Poland,"2Tm2,3","2Tm2,3","2TM2,3",...,0.122266,0.803,0.23076,0.21861,-9.7403,0.06528,143.5114,7.0,False,Christian_metal
3,3,/wiki/List_of_Christian_metal_bands,Marcin Pospieszalski,current,"2Tm2,3",1996–present,Poland,"2Tm2,3","2Tm2,3","2TM2,3",...,0.122266,0.803,0.23076,0.21861,-9.7403,0.06528,143.5114,7.0,False,Christian_metal
4,4,/wiki/List_of_Christian_metal_bands,Dariusz Malejonek,current,"2Tm2,3",1996–present,Poland,"2Tm2,3","2Tm2,3","2TM2,3",...,0.122266,0.803,0.23076,0.21861,-9.7403,0.06528,143.5114,7.0,False,Christian_metal


In [3]:
# Frequency of genres
df.clean_genre.value_counts()

Christian_metal                    3523
early                              2887
metalcore                          2324
glam_metal_bands_and_a             1897
industrial_metal                   1425
doom_metal                         1410
symphonic_metal                    1168
nu_metal                           1105
progressive_metal                   943
grindcore                           638
groove_metal                        631
new_wave_of_British_heavy_metal     591
deathcore                           577
speed_metal                         243
Name: clean_genre, dtype: int64

In [4]:
# List of genres
genres = df.clean_genre.value_counts().index

### Reading in group data

In [5]:
# One row only per group
group_by_group_features = pd.read_csv('final_spotify_df.csv')
group_by_group_features.head()

,Unnamed: 0,Group_name_(wiki),Group_name_(spoti)_x,Spotify_id,Group_popularity,Group_followers,Group_name_(spoti)_y,count_albums,mean_tracks_per_album,total_tracks,...,max_song_popularity,mean_song_danceability,mean_song_acousticness,mean_song_energy,mean_song_instrumentalness,mean_song_liveness,mean_song_loudness,mean_song_speechiness,mean_song_tempo,song_popularity_range
0,0,Krokus,Krokus,spotify:artist:4YveAIZNQZBiWbt7iWsvCa,47.0,223115.0,Krokus,20.0,12.2,244.0,...,52.0,0.5379,0.012151,0.72740,0.242673,0.20389,-9.1140,0.03699,116.2623,18.0
1,1,London,London Symphony Orchestra,spotify:artist:5yxyJsFanEAuwSM5kOuZKc,75.0,233876.0,London Symphony Orchestra,20.0,15.6,312.0,...,62.0,0.3246,0.688700,0.26106,0.560363,0.10833,-16.3342,0.04850,102.3661,14.0
2,2,Thor,Thor,spotify:artist:4DLElWKADc3A99XmSXwCMj,25.0,6157.0,Thor,20.0,15.4,308.0,...,27.0,0.3851,0.079551,0.81700,0.552770,0.20914,-7.4249,0.05866,129.2505,14.0
3,3,Angel Witch,Angel Witch,spotify:artist:6L48AOtGmQL1ELLoKML7cS,36.0,64856.0,Angel Witch,17.0,17.0,289.0,...,49.0,0.2876,0.000659,0.76010,0.112230,0.26206,-10.8665,0.07367,135.1380,29.0
4,4,Ministry,Ministry,spotify:artist:1DXylZlWbVvlckNqwvjTEt,54.0,381254.0,Ministry,20.0,16.6,332.0,...,54.0,0.5668,0.000074,0.88900,0.352359,0.33362,-6.7332,0.05543,130.6994,12.0


In [6]:
for current_genre in genres:

    ###### Creating subset only for a specific genre
    df_genre = df[df['clean_genre']==current_genre]

    ###### Extract only the group names that correspond to a genre
    group_names = list(df_genre['Group_name_(wiki)'].value_counts().index)

    ###### Subset groups within a given genre
    subset_groups_agg_df = group_by_group_features[group_by_group_features['Group_name_(wiki)'].isin(group_names)]
    subset_groups_agg_df['genre'] = current_genre

    ###### Give a number to each group
    subset_groups_agg_df = subset_groups_agg_df.reset_index()
    subset_groups_agg_df = subset_groups_agg_df.drop(['index','Unnamed: 0'], axis=1)
    subset_groups_agg_df['band_number'] = subset_groups_agg_df.index + 1

    ###### Assign band number to each artist
    bands_number_mapping = subset_groups_agg_df[['Group_name_(wiki)', 'band_number']]
    df_genre = df_genre.merge(bands_number_mapping, how='left', on='Group_name_(wiki)')
    df_genre_subset = df_genre[['Group_name_(wiki)', 'attr' , 'origin']].drop_duplicates()
    df_genre_subset = df_genre_subset.groupby('Group_name_(wiki)').head(1)

    ###### Clean origin
    df_genre_subset['origin_clean'] = list(map(lambda x: str(x).split(',')[len(str(x).split(','))-1], 
                                   df_genre_subset['origin']))
    df_genre_subset['origin_clean'] = np.where(df_genre_subset['origin_clean']==' United States', 
                                         ' U.S.', df_genre_subset['origin_clean'])

    ###### Merge back origin to group agg df
    subset_groups_agg_df = subset_groups_agg_df.merge(df_genre_subset, how='left', on='Group_name_(wiki)')

    ###### Produce group aggregate df only for segment
    subset_groups_agg_df = subset_groups_agg_df.fillna(0)
    csv_path_agg_df = str('SEGMENT_group_features_') + str(current_genre) + str('.csv')
    subset_groups_agg_df.to_csv(csv_path_agg_df)

    ##### Identify artists who participate to more than 1 group (serve as connections)
    # Count of different groups per artist
    moved_artists = pd.DataFrame([df_genre.member.value_counts().index, df_genre.member.value_counts().values]).T
    moved_artists = moved_artists.rename({0: 'member', 1: 'count'}, axis='columns')
    # subset of artists who have belonged to multiple groups
    mult_group_artists = moved_artists[moved_artists['count']>1]

    ###### Identify all group connections
    all_groups = []
    for one_member in mult_group_artists.member:

        # Zoom in on all groups for one member
        subset = df_genre[df_genre['member'] == one_member]

        # Find the group numbers for this artist
        groups = list(subset.band_number.value_counts().index)
        groups.sort()
        groups

        for i in range(0, len(groups)):
            for j in range(i+1, (len(groups))):
                # print(str(groups[i]) + '-' + str(groups[j]))
                all_groups.append(str(groups[i]) + '-' + str(groups[j]))

    ##### Summary of connection pairs
    group_pairs_df = pd.DataFrame(all_groups, columns = ['group_pairs'])
    pair = group_pairs_df.group_pairs.value_counts().index 
    count_connections = group_pairs_df.group_pairs.value_counts().values
    group_pairs = pd.DataFrame([pair, count_connections]).T
    group_pairs = group_pairs.rename({0: 'groups', 1: 'count_connections'}, axis='columns')

    #### Finalize pairs df
    group_pairs['groups'] = group_pairs['groups'].astype(str)
    group_pairs['group_1'] = list(map(lambda x: int(x.split('-')[0]), group_pairs['groups']))
    group_pairs['group_2'] = list(map(lambda x: int(x.split('-')[1]), group_pairs['groups']))
    group_pairs_final = group_pairs[['group_1', 'group_2']]

    #### Saving group pairs to csv
    csv_path_group_pairs = str('SEGMENT_group_pairs_') + str(current_genre) + str('.csv')
    group_pairs_final.to_csv(csv_path_group_pairs, index=False)

## Demonstration

Overlap between groups

In [7]:
group_pairs_final

,group_1,group_2
0,12,14
1,11,17


In [9]:
members_band_1 = df_genre[df_genre['band_number']==group_pairs_final['group_1'][0]].member

In [10]:
members_band_2 = df_genre[df_genre['band_number']==group_pairs_final['group_2'][0]].member

In [11]:
set(members_band_1).intersection(members_band_2)

{' Michael Ehré', 'Ralf Scheepers'}

In [12]:
current_genre

'speed_metal'